# M7003_finalpiece_24000114067

LIS MASc  
The Right Word  
Final Piece (Choice 2: NLP)  
Student number: 24000114067

Access notebook, data, pdf and web app in the [GitHub repository](https://github.com/noah-art3mis/nlp-assignment).

PDF generated using [nbconvert](https://nbconvert.readthedocs.io/en/latest/)

## Intersect - Personalized job matching

This is an web app (acess on [streamlit]()) that can be used to find jobs that match your profile.

similar in concept to [marginalia](https://marginalia-search.com/) in that it helps users to find nonobvious results buried deep into the ordering given by a website.

-   [ ] The NLP-aided analysis of a corpus of documents will extract non-obvious insights from a corpus of documents using tools from coding and data science. The corpus of documents could consist of a literary archive, a body of social media posts, a scrape from an online knowledge database; ideally, these would have some relation to the capstone problem. The submission should offer a graphical representation of its discoveries and include a write up of c. 500 words the documents the process involved. A link to an executable code notebook should also be included as part of the submission.
-   [ ] Assessment Choice 2 should be submitted as a PDF file that contains a brief summary of the submission and which contains a link to a GitHUb repository where the code notebook and all relevant data can be found - max 500 words

ai sells. while i find it intellectually dishonest to call reordering a table using embeddings 'intelligent', in the current zeitgeist i believe this is the play. i have seen far worse.

-   findings
-   illustrations

this is not necessary related to the capstone but is relevant for my future and can also help other people and also be a piece for a portfolio. this can become an actual product if it is good.

the data is publicly available and their robots.txt file does not disallow scraping of these pages.

while we're here, we can also do some basic nlp to see if theres anything there. we already have the data.

more details regarding the choices made can be found in the README file.

## comparing the results from semantic to lexical search in a subjective and qualitative manner we get that X. for a more serious project this analysis would need to be automated with benchmarks and evals and so on.

This work contains [ ] words (not counting tables and figures).

---

-   The submission should use technical concepts from linguistics and/or NLP
    in practically useful way, drawing on relevant sources and tools.
-   The submission should justify the intellectual choices it makes with respect
    to analysis and/or optimisation in relation to theoretical methods and
    established methodologies—and do so in a critically informed way.
-   Where relevant, the submission should convincingly manipulate language
    and/or narrative qualitatively and/or quantitatively, using appropriate
    methods.
-   The assessment should evidence skills in the communication of complex
    ideas in its chosen format.



## Explanations

-   the idea is to intersect two searches - the original one with keywords and the semantic one with text. this makes it so you can search both for what you want and what you need at the same time. you can do keyword as a want (`fun`) or as need (`data`) and vice versa.
-   OpenAI, Streamlit and Cohere are the fastest ways to make a prototype. This is the same reason there is no need for a vector database or even sqlite.
-   For similarity we can use a dot product because OpenAI embeddings are normalized.
-   _CV-Library_ is used as the data source as it is cited by the [UK Government](https://nationalcareers.service.gov.uk/careers-advice/advertised-job-vacancies).
-   Scraping
    -   Respects robots.txt
    -   just a request doesnt work (403)
    -   request with user agent doesnt work (403)
    -   request with genuine headers doesnt work (403)
    -   request with scrapeops fake headers doesnt work (403)
    -   request with scrapeops proxy works but takes like 20s per request. this is tolerable for this application. scrapeops is free.
-   test cvs generated by chatgpt with the prompt 'generate the perfect cv for this position' get around 0.65 similarity.
-   BM25 is an altered version of TF-IDF that is standard in the industry.
    -   https://www.elastic.co/blog/practical-bm25-part-2-the-bm25-algorithm-and-its-variables
    -   https://docs.llamaindex.ai/en/stable/examples/retrievers/bm25_retriever/
    -   bm25f can weight different parameters as relevant but at this point you are replicating the job board and that is out of scope.
    -   the pdf/text input can be anything that you like. search for a job that is like a song or a poem or something. go nuts
    -   note that the similarity values do not mean much of anything in absolute. see
        -   https://datascience.stackexchange.com/questions/101862/cosine-similarity-between-sentence-embeddings-is-always-positive
        -   https://vaibhavgarg1982.medium.com/why-are-cosine-similarities-of-text-embeddings-almost-always-positive-6bd31eaee4d5
    -   much like llms, embeddings do averages. and so this is going to be generic. if you are trying to do something fancy, you might want to use this in a nonstandard way as well. such as using weird keywords in the search.
    -   biggest issue time-wise is the free proxies. just buy good ones. also less of a security risk.
    -   for a production-grade implementation of this we might want to take a look at https://github.com/AmenRa/retriv
    -   Elastic search setup is a bit more involved so out of scope
    -   why pca and kmeans

bm25 is is an industry standard algorithm for search. bm25 itself is a variation on tfidf. this uses the lucene method as it is the default of the bm25s package.

can also add a reranker after the semantic search

-   cohere rerank
-

measure how much time it takes to use a reranker and ebedding local and remote.

lexical search. sparse vectors
semantic search. dense vectors.
